#### Підготовка середовища

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt
import numpy as np
import os

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

ImportError: Traceback (most recent call last):
  File "c:\Users\anast\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

#### Завантажимо дані у TensorFlow:

In [ ]:
train_dir = "101_food_classes_10_percent/train_s"
test_dir = "101_food_classes_10_percent/test_s"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='categorical',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    #subset='training',
    seed=42
)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='categorical',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

print("Класи, які завантажено:", train_data.class_names)
print(train_data.class_names)
print(test_data.class_names)

#### Побудова моделі CNN

In [ ]:

model = models.Sequential([
   layers.Input(shape=(224,224,3)),
   layers.Rescaling(1./255),
   layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
   layers.MaxPooling2D(2,2),

   layers.Conv2D(64, (3,3), activation='relu'),
   layers.MaxPooling2D(2,2),

   layers.Flatten(),
   layers.Dense(128, activation='relu'),
   layers.Dropout(0.5),

   layers.Dense(3, activation='softmax')
])
model.summary()

#### Компіляція моделі

In [ ]:

model.compile(
   optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
   loss='categorical_crossentropy',
   metrics=['accuracy']
)

#### Навчання моделі

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
   monitor='val_accuracy',
   patience=5,
   restore_best_weights=True
)

history = model.fit(
   train_data,
   validation_data=test_data,
   epochs=20,
   callbacks=[early_stop]
)

#### Аналіз результатів

In [ ]:
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

#### Оцінка моделі

In [ ]:
loss, acc = model.evaluate(test_data)
print(f"Test accuracy: {acc:.3f}")

#### Приклади передбачень

In [ ]:
class_names = train_data.class_names

for images, labels in test_data.take(1):
   preds = model.predict(images)
   for i in range(10):
      plt.imshow(images[i].numpy().astype("uint8"))
      plt.title(f"True: {class_names[np.argmax(labels[i])]}\nPred: {class_names[np.argmax(preds[i])]} — {preds[i]}")
      # plt.title(f"Pred: {class_names[np.argmax(preds[i])]} — {preds[i]}")
      plt.axis("off")
      plt.show()